## 1G Pick Planning Model

### Import Libraries & Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from utility.data_puller import DataPuller
from utility.config_loader import load_config
from utility.logging_config import setup_logging
from utility.data_validator import DataValidator
from sim_engine import SimEngine


### Fetch Input Data

In [ ]:
# Load configuration
main_dir = Path().resolve()
config_path = os.path.join(main_dir.parent, 'config', 'config.yaml')

config = load_config(config_path)

# Set up logging
logger = setup_logging(config)

input_mode = config['global']['input_mode']

# Initialize DataPuller
data_puller = DataPuller(config_path, input_mode)

# Get global parameters used when fetching data
fc = config['global']['wh_id']
start_time = config['global']['start_time']
end_time = config['global']['end_time']

#Pull data
logger.info("Pulling required data...")
container_data = data_puller.get_container_data(fc, start_time, end_time)
slotbook_data = data_puller.get_slotbook_data(fc)



### Input Data Validation

In [ ]:
# Initialize validator
validator = DataValidator(config)

# Validate and analyze data
container_data, slotbook_data = validator.validate(container_data, slotbook_data)


### Run Pick Planning Iteratively

In [ ]:
# Initialize engine
engine = SimEngine(config)

# Run simulation
results = engine.run(
    container_data=container_data,
    slotbook_data=slotbook_data,
    start_time=start_time,
    end_time=end_time
)

# Access results
consolidated_results = results.get_consolidated_results()


In [ ]:
consolidated_results['container_assignments'].to_csv('container_assignments.csv')
